# Needed imports and utility

In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from comtypes.client import CreateObject

import ansys.speos.core as core
from ansys.speos.core.speos import Speos
import ansys.speos.workflow as workflow

tests_data_path defines the asset files location, where pre-prepared files are used in this demonstration.
In this simple demonstration, we have prepared one triangle prism geometry and flat surface source.
Of course, you can also use or create your own geometry from mesh information.

In [2]:
tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

Some utility functions are defined at the beginning:
function print_message: Print rpc simulation progress message.
function open_file: Open and display the simulation result.

In [3]:
def open_file(file):
    dpf_instance = None
    if file.endswith("xmp") or file.endswith("XMP"):
        dpf_instance = CreateObject("XMPViewer.Application")
        dpf_instance.OpenFile(file)
        res = dpf_instance.ExportXMPImage(file+".png", 1)
        if res:
            img = mpimg.imread(file+".png")
            plt.imshow(img)
            plt.axis("off")   # turns off axes
            plt.axis("tight")  # gets rid of white border
            plt.axis("image")  # square up the image instead of filling the "figure" space
            plt.show()
    elif file.endswith("hdr") or file.endswith("HDR"):
        dpf_instance = CreateObject("HDRIViewer.Application")
        dpf_instance.OpenFile(file)
        dpf_instance.Show(1)
    elif file.endswith("png") or file.endswith("PNG"):
        img = mpimg.imread(file)
        plt.imshow(img)
        plt.axis("off")   # turns off axes
        plt.axis("tight")  # gets rid of white border
        plt.axis("image")  # square up the image instead of filling the "figure" space
        plt.show()
    return dpf_instance

def print_message(msg):
    print(core.protobuf_message_to_str(msg))

# Create connection with speos rpc server
In this simple demonstration, a local host server is used. However, such server can be a remote server or can be on cloud.

Firstly, a speos client is initiated.
This speos client will later provide all the necessary rpc API for us to create workflow.

After a speos client is created, it is always best practice to clean all the database inside.
clean_all_dbs function from workflow class is defined to clean all the database in the memory.

In [6]:
speos = Speos(host="localhost", port=50098)
workflow.clean_all_dbs(speos.client)

# Create SpeosSimulationUpdate
After a client is created and cleaned, it is empty, i.e. there is nothing inside.
Thus, we need add different objects.
Just like how we see the world, there are
* sources: sun, lamp, etc
* objects: table, food, street, and buildings, etc
* detectors: our eye or cameras
SpeosSimulationupdate is an object which is initialized to host all the necessary information required to run a simulation.
Firstly, a pre-prepared SpeosSimulationUpdate, named my_scene_simu, is initialized.

In [7]:
sim_file = os.path.join(tests_data_path, "Env_Simplified_Empty.speos", "Env_Simplified_Empty.speos")
my_scene_simu = workflow.SpeosSimulationUpdate(speos, sim_file, clean_dbs=False)

Can use .scene.get() to see what has been included.
In this pre-prepared, a sunlight environment has been added.

In [8]:
print(my_scene_simu.scene.get())

ansys.api.speos.scene.v2.Scene
{
    "name": "Env_Simplified_Empty",
    "description": "From D:\\Work\\Gitdir\\pyspeos\\tests\\jupyter_notebooks\\..\\assets\\Env_Simplified_Empty.speos\\Env_Simplified_Empty.speos",
    "part_guid": "db847e53-64b8-48e0-a7b7-ee4b439f5ee3",
    "sensors": [
        {
            "name": "Irradiance.1:51",
            "sensor_guid": "89c2d2de-7c9b-4221-b1a0-ddfdbf32f462",
            "result_file_name": "Env_Simplified.Irradiance.1",
            "irradiance_properties": {
                "axis_system": [
                    0.0,
                    0.0,
                    0.0,
                    1.0,
                    0.0,
                    0.0,
                    0.0,
                    1.0,
                    0.0,
                    0.0,
                    0.0,
                    1.0
                ],
                "layer_type_none": {},
                "ray_file_type": "RayFileNone",
                "integration_direction": []
          

Then, we would like to add more objects: road

In [9]:
env_file = os.path.join(tests_data_path, "Env_Simplified.speos", "Env_Simplified.speos")
env_scene = workflow.SpeosSimulationUpdate(speos, env_file, clean_dbs=False)

env_position = core.AxisSystem()
my_scene_simu.add_scene(simulation_scene=env_scene, position_info=env_position, only_geometry=True)
my_scene_simu.preview()

Widget(value='<iframe src="http://localhost:62546/index.html?ui=P_0x2713aa2a2c0_0&reconnect=auto" class="pyvis…

Add more objects: cars.
Cars' position information are required before adding.
Position are defined by origin and x, y, z axis vectors.
The unit used is mm by default. You can use preview function to check how different origin coordinate and x, y, z vectors change its positions.

In [ ]:
blue_car_file = os.path.join(tests_data_path, "BlueCar.speos", "BlueCar.speos")
red_car_file = os.path.join(tests_data_path, "RedCar.speos", "RedCar.speos")

blue_car_scene = workflow.SpeosSimulationUpdate(speos, blue_car_file, clean_dbs=False)
red_car_scene = workflow.SpeosSimulationUpdate(speos, red_car_file, clean_dbs=False)

blue_car_position = core.AxisSystem()
blue_car_position.origin = [1984.87, 0, 34660.06]
blue_car_position.x_vect = [0.0, 0.0, -1.0]
blue_car_position.y_vect = [-1.0, 0.0, 0.0]
blue_car_position.z_vect = [0.0, 1.0, 0.0]

red_car_position = core.AxisSystem()
red_car_position.origin = [-3749, 0.0, 48000]
red_car_position.x_vect = [1.0, 0.0, 0.0]
red_car_position.y_vect = [0.0, 0.0, -1.0]
red_car_position.z_vect = [0.0, 1.0, 0.0]

my_scene_simu.add_scene(simulation_scene=red_car_scene, position_info=red_car_position, only_geometry=True)
my_scene_simu.add_scene(simulation_scene=blue_car_scene, position_info=blue_car_position, only_geometry=True)
my_scene_simu.preview()

# Add a camera sensor to the scene
To simulate the car camera, a camera detector is added.
Just like cameras in the real word, there are different types of camera, e.g. one with long focal distance, one with wider field of view, etc.
There are several parameters settings required when defining a camera:
Transmittance: defines how different wavelength light will travel through the camera lens
Distortion: how light will be distorted at different position after travelling through the lens. Example can be found on website: https://en.wikipedia.org/wiki/Distortion_(optics)
Color_spectrum: defines how sensitive a camera can detector a red / blue / green light
Wavelengths: defines the range of wavelength such camera can detector. For most camera, visual wavelength range used. There are some camera designed for infrared which means its wavelength range cover the infrared part.
Focal_length: defines focal distance of a camera
Pixel and dimension: define the camera size and resolution

In [ ]:
camera_input_path = os.path.join(tests_data_path, "CameraInputFiles")
camera_parameters = workflow.PhotometricCameraSensorParameters()
camera_parameters.name = "Camera_Sensor"
camera_parameters.transmittance_file = os.path.join(camera_input_path, "CameraTransmittance.spectrum")
camera_parameters.distortion_file = os.path.join(camera_input_path, "CameraDistortion_190deg.OPTDistortion")
camera_parameters.color_mode_red_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityRed.spectrum")
camera_parameters.color_mode_green_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityGreen.spectrum")
camera_parameters.color_mode_blue_spectrum_file = os.path.join(camera_input_path, "CameraSensitivityBlue.spectrum")
camera_parameters.wavelengths_start = 380
camera_parameters.wavelengths_end = 780
camera_parameters.wavelengths_sampling = 21
camera_parameters.focal_length = 7
camera_parameters.imager_distance = 1
camera_parameters.f_number = 20
camera_parameters.horizontal_pixel = 964
camera_parameters.vertical_pixel = 544
camera_parameters.width = 8.0976
camera_parameters.height = 4.5696

camera_props = workflow.CameraSensorProperties()
camera_props.origin = [-2118.67, 1500, 11000]
camera_props.x_vect = [-0.999694, 0.0, -0.0247255]
camera_props.y_vect = [0.0, 1.0, 0.0]
camera_props.z_vect = [0.0, 0.0, -1.0]

my_scene_simu.add_camera_sensor(camera_parameters, camera_props)

# Compute the first simulation

In [ ]:
job_link = my_scene_simu.compute("first_job")
print_message(job_link.get_results())

# Open and view the result file

In [ ]:
speos_result_file = job_link.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link.get_results().results[1].path
open_file(speos_result_file)

In [ ]:
job_link.delete()

# Add more environment lights
In addition to environment light, there are a lot of light sources, e.g. from traffic light, etc.
Here we are adding traffic light.
This time, in addition to geometry, the light source inside are also added.
add_scene method without only_geometry as True would add light sources inside.

In [ ]:
traffic_light_file = os.path.join(tests_data_path, "TrafficLight.speos", "TrafficLight.speos")
traffic_light_scene = workflow.SpeosSimulationUpdate(speos, traffic_light_file, clean_dbs=False)

my_scene_simu.add_scene(traffic_light_scene, env_position)
my_scene_simu.preview()

# Compute the second simulation

In [ ]:
job_link_2 = my_scene_simu.compute("second_job")
print_message(job_link_2.get_results())

Open and view the result file

In [ ]:
speos_result_file = job_link_2.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link_2.get_results().results[1].path
open_file(speos_result_file)

In [ ]:
job_link_2.delete()

# Move objects
objects change their positions in reality.
update_scene_part_position method is used to update object position.

In [ ]:
red_car_position_update = red_car_position
red_car_position_update.origin = [0.0, 0.0, 48000]
blue_car_position_update = blue_car_position
blue_car_position_update.origin = [1984.87, 0, 30000]

new_body_positions = {
    red_car_scene.scene.get().name: red_car_position_update,
    blue_car_scene.scene.get().name: blue_car_position_update,
}
my_scene_simu.update_scene_part_position(new_body_positions)

Here are simulate a number of different positions to simulate a car moving on the road:
Depending on the speed of car and frame number second of camera, the new position can be calculated:
Let's assume the camera is 24 frames per second and car is moving at 30 miles per hour (13.4112 meters by second)
Thus, between each frame, the distance is about 0.5m or 500 mm.

In [ ]:
blue_car_position_update = blue_car_position
new_body_positions = {
    blue_car_scene.scene.get().name: blue_car_position_update,
}
for frame in range(5):
    blue_car_position_update.origin = [1984.87, 0, 30000 - 500 * frame ]
    new_body_positions[blue_car_scene.scene.get().name] = blue_car_position_update
    my_scene_simu.update_scene_part_position(new_body_positions)
    job_link = my_scene_simu.compute("road_simulation_{}".format(frame))
    print_message(job_link.get_results())

# Compute the modified simulation with GPU
Note: a student license does not support GPU compute

In [ ]:
job_link2 = my_scene_simu.compute("second_job", compute_type="gpu")
print_message(job_link2.get_results())

# Open and view the modified result file

In [ ]:
speos_result_file = job_link2.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link2.get_results().results[1].path
open_file(speos_result_file)

In [ ]:
job_link2.delete()

# Move the camera position
Similarly, camera will also move.
update_sensor method is used to update the position of camera.

In [ ]:
camera_props_update = workflow.CameraSensorProperties()
camera_props_update.origin = [-2118.67, 1500.0, 21000.0]
camera_props_update.x_vect = [-0.999694, 0.0, -0.0247255]
camera_props_update.y_vect = [0.0, 1.0, 0.0]
camera_props_update.z_vect = [0.0, 0.0, -1.0]

my_scene_simu.update_sensor(camera_parameters, camera_props_update)

# Compute the modified simulation
Note: a student license does not support GPU compute

In [ ]:
job_link3 = my_scene_simu.compute("third_job", compute_type="gpu")
print_message(job_link3.get_results())

# Open and view the modified simulation result files

In [ ]:
speos_result_file = job_link3.get_results().results[4].path
open_file(speos_result_file)
speos_result_file = job_link3.get_results().results[1].path
open_file(speos_result_file)

In [ ]:
job_link3.delete()

# Clean after usage

In [ ]:
my_scene_simu.close()